# 07 — Marketing Report: Channel Performance & Budget Insights

Marketing Mix Modeling (MMM) uses statistical inference to estimate how much each marketing channel contributes to sales, accounting for lagged effects, diminishing returns, and external factors like promotions and customer satisfaction. This report summarises the model's findings for the DT Mart dataset (Jul 2015 – Jun 2016) across four channels: TV, Sponsorship, Digital, and Online.

**What this report covers:** spend allocation versus contribution to GMV, how long campaigns keep working after they end, where diminishing returns are already visible, and a directional view of where rebalancing the budget could improve returns.

> **Data note:** This dataset covers one year of weekly data with monthly media spend distributed uniformly across weeks. Model estimates are directional. Treat channel rankings as guidance, not precise measurements.

---
## 1. Setup

In [ ]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.special import expit

warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid")
%matplotlib inline

from pymc_marketing.mmm import MMM

from mmm_demo.config import OUTPUTS_DIR, ModelConfig
from mmm_demo.data import load_mmm_weekly_data
from mmm_demo.diagnostics import check_convergence
from mmm_demo.model import sample_posterior_predictive

CHANNEL_COLORS = {
    "TV": "#4C72B0",
    "Sponsorship": "#DD8452",
    "Digital": "#55A868",
    "Online": "#C44E52",
}

---
## 2. Load Results

> **Prerequisite:** Run notebooks 02 and 03 first. This notebook loads the most recent fitted model from `outputs/models/`.

In [ ]:
# --- Load weekly data ---
df = load_mmm_weekly_data()
config = ModelConfig()
feature_cols = [config.date_column, *config.channel_columns, *config.control_columns]
x = df[feature_cols]
y = df[config.target_column]

# --- Load most recent fitted model ---
model_dir = OUTPUTS_DIR / "models"
model_files = sorted(model_dir.glob("mmm_fit_*.nc"))
if not model_files:
    raise FileNotFoundError("No saved model found. Run notebook 02 first.")

model_path = model_files[-1]
print(f"Loading model: {model_path.name}")
mmm = MMM.load(str(model_path))
idata = mmm.idata
print(
    f"Model loaded. Chains: {idata.posterior.dims['chain']}, "
    f"Draws: {idata.posterior.dims['draw']}"
)

# --- Load contribution summary table if available ---
tables_dir = OUTPUTS_DIR / "tables"
contrib_summary_csv = None
try:
    csv_files = sorted(tables_dir.glob("contribution_summary_*.csv"))
    if csv_files:
        contrib_summary_csv = pd.read_csv(csv_files[-1], index_col=0)
        print(f"Loaded contribution summary: {csv_files[-1].name}")
    else:
        print("No contribution summary CSV found — will compute from model.")
except Exception:
    print("Could not load contribution summary CSV — will compute from model.")

# --- Load ROI table if available ---
opt_dir = OUTPUTS_DIR / "optimization"
roi_csv = None
try:
    roi_files = sorted(opt_dir.glob("channel_roi_*.csv"))
    if roi_files:
        roi_csv = pd.read_csv(roi_files[-1], index_col=0)
        print(f"Loaded ROI table: {roi_files[-1].name}")
    else:
        print("No ROI CSV found — will compute from model.")
except Exception:
    print("Could not load ROI CSV — will compute from model.")

In [ ]:
# Data at a glance
total_annual_gmv = y.sum()
total_annual_spend = df[config.channel_columns].sum().sum()
date_range = f"{df[config.date_column].min().strftime('%d %b %Y')} to {df[config.date_column].max().strftime('%d %b %Y')}"
n_weeks = len(df)

glance = pd.DataFrame(
    {
        "Metric": [
            "Date range",
            "Number of weeks",
            "Total annual GMV (INR)",
            "Total annual media spend (INR)",
        ],
        "Value": [
            date_range,
            n_weeks,
            f"{total_annual_gmv:,.0f}",
            f"{total_annual_spend:,.0f}",
        ],
    }
).set_index("Metric")

print("Data at a glance:")
glance

### Model Reliability Check

Before presenting results, we check whether the model estimates are reliable. A well-behaved model produces stable, consistent estimates across independent runs. An unreliable model produces estimates driven more by modeling assumptions than by the data — in that case, findings should be treated as directional only.

In [ ]:
diag = check_convergence(idata)

reliability_label = "RELIABLE" if diag.passed else "DIRECTIONAL ONLY"
print(f"Model reliability: {reliability_label}")
print()

if diag.passed:
    print("The model has converged — estimates reflect the data well.")
else:
    print(
        "The model estimates are influenced by modeling assumptions alongside the data."
    )
    print(
        "This is expected with only 12 distinct monthly media inputs across 52 weeks."
    )
    print(
        "Channel rankings are directional. Do not use for precise budget commitments."
    )
    print()
    print(
        "See notebook 03 for a detailed explanation of this limitation and next steps."
    )

---
## 3. Current Media Mix

The chart below shows how the annual media budget is currently distributed across the four channels. Sponsorship is the single largest channel at 46.6% of total spend, followed by Online (31.5%), Digital (16.3%), and TV (5.6%).

In [ ]:
current_spend = df[config.channel_columns]
mean_weekly_spend = current_spend.mean()
spend_share = (mean_weekly_spend / mean_weekly_spend.sum() * 100).round(1)
total_weekly_budget = mean_weekly_spend.sum()
channel_max_spend = current_spend.max()

colors = [CHANNEL_COLORS[ch] for ch in config.channel_columns]

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Bar: mean weekly spend
mean_weekly_spend.plot.bar(ax=axes[0], color=colors, edgecolor="black", linewidth=0.5)
axes[0].set_title("Mean Weekly Spend per Channel", fontsize=12, fontweight="bold")
axes[0].set_ylabel("INR")
axes[0].tick_params(axis="x", rotation=0)
for bar, val in zip(axes[0].patches, mean_weekly_spend, strict=False):
    axes[0].text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() * 1.02,
        f"{val/1e6:.1f}M",
        ha="center",
        fontsize=9,
    )

# Pie: spend share
wedge_props = {"edgecolor": "white", "linewidth": 1.5}
spend_share.plot.pie(
    ax=axes[1],
    autopct="%1.1f%%",
    colors=colors,
    startangle=90,
    wedgeprops=wedge_props,
    pctdistance=0.75,
)
axes[1].set_title("Spend Share (%)", fontsize=12, fontweight="bold")
axes[1].set_ylabel("")

plt.suptitle("Current Media Budget Allocation", fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

print("Current spend allocation (weekly average):")
for ch in config.channel_columns:
    print(
        f"  {ch:<15} {mean_weekly_spend[ch]/1e6:>6.1f}M INR/week  ({spend_share[ch]:.1f}%)"
    )
print(f"  {'TOTAL':<15} {total_weekly_budget/1e6:>6.1f}M INR/week")

Sponsorship is the largest channel at 46.6% of total spend — nearly half the media budget. Online is second at 31.5%, making the two channels together responsible for 78% of spend. TV is the smallest channel, receiving just 5.6% of the budget. This concentration in two channels raises a key question: do the contribution numbers justify this allocation?

---
## 4. Channel Contribution to GMV

The model breaks down total GMV into the estimated contribution of each component: baseline (organic sales without any marketing), each marketing channel, and control variables like discounts and special sale events. The charts below show these contributions in original GMV units.

In [ ]:
# Ensure posterior predictive is available for the model plots
if "posterior_predictive" not in list(idata.groups()):
    print("Sampling model predictions (needed for contribution plots)...")
    sample_posterior_predictive(mmm, x)
    idata = mmm.idata
    print("Done.")

fig = mmm.plot_waterfall_components_decomposition(original_scale=True, figsize=(14, 7))
plt.suptitle(
    "What Drives GMV? — Contribution of Each Component (Annual Average)",
    fontsize=13,
    fontweight="bold",
    y=1.01,
)
plt.tight_layout()
plt.show()

In [ ]:
fig = mmm.plot_components_contributions()
if hasattr(fig, "suptitle"):
    fig.suptitle(
        "How Did Each Channel Perform Over the Year?",
        fontsize=13,
        fontweight="bold",
        y=1.01,
    )
plt.tight_layout()
plt.show()

In [ ]:
# Compute mean contributions over time
contributions = mmm.compute_mean_contributions_over_time(original_scale=True)
avg_contributions = contributions.mean()
total_contrib = avg_contributions.sum()

# Channel-only contributions
channel_contrib = avg_contributions[config.channel_columns]
channel_contrib_share = (channel_contrib / channel_contrib.sum() * 100).round(1)

fig, ax = plt.subplots(figsize=(9, 5))
colors = [CHANNEL_COLORS[ch] for ch in config.channel_columns]

bars = ax.bar(
    config.channel_columns,
    channel_contrib.values,
    color=colors,
    edgecolor="black",
    linewidth=0.5,
)
for bar, val, share in zip(
    bars, channel_contrib.values, channel_contrib_share.values, strict=False
):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() * 1.02,
        f"{val/1e6:.1f}M\n({share:.0f}% of channel GMV)",
        ha="center",
        fontsize=9,
    )

ax.set_title(
    "Average Weekly GMV Contribution per Channel (Marketing Channels Only)",
    fontsize=12,
    fontweight="bold",
)
ax.set_ylabel("INR (average weekly)")
ax.tick_params(axis="x", rotation=0)
plt.tight_layout()
plt.show()

print("Average weekly channel contributions:")
for ch in config.channel_columns:
    print(
        f"  {ch:<15} {channel_contrib[ch]/1e6:>6.1f}M INR  ({channel_contrib_share[ch]:.1f}% of channel GMV)"
    )
print()
print(
    "Note: Shaded bands on the time-series plots above represent the model's confidence range"
    " — wider bands mean more uncertainty."
)

---
## 5. Spend vs Contribution: Are We Over- or Under-Investing?

Comparing spend share to contribution share reveals where the budget is working hardest. A channel with a higher contribution share than spend share is delivering outsized returns — it may be under-invested. A channel where spend share exceeds contribution share is delivering diminishing returns and may be a candidate for reallocation.

In [ ]:
contrib_share_pct = channel_contrib_share.rename("Contribution Share (%)")
spend_share_renamed = spend_share.rename("Spend Share (%)")

comparison_df = pd.DataFrame(
    {
        "Spend Share (%)": spend_share_renamed,
        "Contribution Share (%)": contrib_share_pct,
    }
)
comparison_df["Difference (Contrib - Spend)"] = (
    comparison_df["Contribution Share (%)"] - comparison_df["Spend Share (%)"]
).round(1)

fig, ax = plt.subplots(figsize=(10, 5))
x_pos = np.arange(len(config.channel_columns))
width = 0.35

bars1 = ax.bar(
    x_pos - width / 2,
    comparison_df["Spend Share (%)"],
    width,
    label="Spend Share",
    color="#B0C4DE",
    edgecolor="black",
    linewidth=0.5,
)
bars2 = ax.bar(
    x_pos + width / 2,
    comparison_df["Contribution Share (%)"],
    width,
    label="Contribution Share",
    color=[CHANNEL_COLORS[ch] for ch in config.channel_columns],
    edgecolor="black",
    linewidth=0.5,
)

ax.set_xticks(x_pos)
ax.set_xticklabels(config.channel_columns)
ax.set_ylabel("Share of total (%)")
ax.set_title(
    "Spend Share vs Contribution Share by Channel",
    fontsize=12,
    fontweight="bold",
)
ax.axhline(0, color="black", linewidth=0.8)
ax.legend()

# Annotate the difference
for i, ch in enumerate(config.channel_columns):
    diff = comparison_df.loc[ch, "Difference (Contrib - Spend)"]
    color = "#2a7a2a" if diff > 0 else "#a01010"
    ax.annotate(
        f"{diff:+.1f}pp",
        xy=(
            i,
            max(
                comparison_df.loc[ch, "Spend Share (%)"],
                comparison_df.loc[ch, "Contribution Share (%)"],
            )
            + 1,
        ),
        ha="center",
        fontsize=9,
        color=color,
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

print("Spend vs Contribution comparison (percentage points):")
print(comparison_df.to_string())
print()
print(
    "Positive difference = contributes more than its spend share (potentially under-invested)"
)
print(
    "Negative difference = contributes less than its spend share (potentially over-invested)"
)

The comparison reveals an important pattern: channels that receive more budget are not necessarily the ones generating proportionally more GMV. Channels with a positive difference (contribution share exceeds spend share) are delivering more GMV per rupee spent and may warrant increased investment. Channels with a negative difference are showing early signs of diminishing returns — each additional rupee is generating less incremental GMV.

**Caveat:** With only 12 months of data, these contribution estimates carry substantial uncertainty. The confidence ranges on the time-series charts reflect this. Treat the direction of each gap as a signal, not a precise measurement. The saturation analysis in Section 7 provides additional evidence.

---
## 6. Campaign Carryover: How Long Do Campaigns Keep Working?

When you run a campaign this week, it does not stop working when the week ends. The brand impression, the recall, the click that turned into a delayed purchase — all of these carry over into future weeks. The model estimates how much of this week's advertising effect persists into the following weeks for each channel.

The carryover rate is the fraction of the current week's effect that spills into the next. A rate of 0.30 means 30% of this week's effect is still active next week, 9% the week after (0.30 squared), and so on.

In [ ]:
summary_alpha = az.summary(idata, var_names=["adstock_alpha"])

# Build carryover table
carryover_rows = []
for ch in config.channel_columns:
    param = f"adstock_alpha[{ch}]"
    alpha_mean = summary_alpha.loc[param, "mean"]
    alpha_lo = summary_alpha.loc[param, "hdi_3%"]
    alpha_hi = summary_alpha.loc[param, "hdi_97%"]
    carryover_rows.append(
        {
            "Channel": ch,
            "Carryover rate (mean)": round(alpha_mean, 3),
            "Confidence range": f"{alpha_lo:.2f} – {alpha_hi:.2f}",
            "Effect at week 1 (%)": f"{alpha_mean * 100:.0f}%",
            "Effect at week 2 (%)": f"{alpha_mean**2 * 100:.0f}%",
            "Effect at week 4 (%)": f"{alpha_mean**4 * 100:.0f}%",
        }
    )

carryover_df = pd.DataFrame(carryover_rows).set_index("Channel")
print("Campaign carryover by channel:")
print(carryover_df.to_string())
print()
print(
    "Interpretation: a channel with 30% carryover at week 1 means 30% of this week's"
    " campaign effect is still active in the following week."
)

In [ ]:
k_max = config.adstock_max_lag
weeks = np.arange(0, k_max + 1)

fig, ax = plt.subplots(figsize=(10, 5))

for ch in config.channel_columns:
    param = f"adstock_alpha[{ch}]"
    alpha_mean = summary_alpha.loc[param, "mean"]
    alpha_lo = summary_alpha.loc[param, "hdi_3%"]
    alpha_hi = summary_alpha.loc[param, "hdi_97%"]

    decay_mean = alpha_mean**weeks * 100
    decay_lo = alpha_lo**weeks * 100
    decay_hi = alpha_hi**weeks * 100

    ax.plot(
        weeks,
        decay_mean,
        color=CHANNEL_COLORS[ch],
        linewidth=2.5,
        label=f"{ch} (carryover rate = {alpha_mean:.2f})",
        marker="o",
        markersize=6,
    )
    ax.fill_between(weeks, decay_lo, decay_hi, alpha=0.12, color=CHANNEL_COLORS[ch])

ax.set_xlabel("Weeks after campaign ends", fontsize=11)
ax.set_ylabel("% of original effect still active", fontsize=11)
ax.set_title(
    "Campaign Carryover: How Long Does Each Channel Keep Working?",
    fontsize=12,
    fontweight="bold",
)
ax.legend(loc="upper right")
ax.set_ylim(0, 105)
ax.set_xticks(weeks)
ax.set_xticklabels([f"Week {int(w)}" if w > 0 else "Campaign week" for w in weeks])
plt.tight_layout()
plt.show()

The decay curves show how quickly each channel's effect fades after a campaign ends. Channels with a higher carryover rate (a slower-falling curve) deliver a longer-lasting impact — spending today continues to contribute to GMV for several weeks. This has implications for budget timing: channels with strong carryover can tolerate brief spend pauses without losing much cumulative impact, while channels with rapid decay need consistent investment to maintain effect.

The shaded bands represent the model's confidence range. Wider bands indicate more uncertainty — with only 12 months of data, the model cannot always distinguish a fast-decay channel from a slow-decay one with high confidence. The confidence ranges should be considered when drawing conclusions about specific channels.

---
## 7. Diminishing Returns: Are We in the Saturation Zone?

Every channel has a point beyond which spending more produces less incremental GMV. Early in a channel's spend range, each additional rupee generates substantial sales — the curve is steep. At higher spend levels, the curve flattens — you are getting fewer sales per rupee. This is **diminishing returns**.

The chart below shows the estimated response curve for each channel. The vertical line marks the channel's current average weekly spend level. A dot in the steep part of the curve means the channel still has room to grow. A dot on the flat part signals saturation.

In [ ]:
summary_lam = az.summary(idata, var_names=["saturation_lam"])
summary_beta = az.summary(idata, var_names=["saturation_beta"])

x_scaled = np.linspace(0, 1, 300)  # 0 = zero spend, 1 = maximum observed spend

fig, axes = plt.subplots(1, len(config.channel_columns), figsize=(14, 5), sharey=False)

saturation_interpretation = []

for i, ch in enumerate(config.channel_columns):
    lam_param = f"saturation_lam[{ch}]"
    beta_param = f"saturation_beta[{ch}]"

    lam_mean = summary_lam.loc[lam_param, "mean"]
    lam_lo = summary_lam.loc[lam_param, "hdi_3%"]
    lam_hi = summary_lam.loc[lam_param, "hdi_97%"]
    beta_mean = summary_beta.loc[beta_param, "mean"]

    # PyMC-Marketing LogisticSaturation: beta * (2 * expit(lam * x) - 1)
    y_mean = beta_mean * (2 * expit(lam_mean * x_scaled) - 1)
    y_lo = beta_mean * (2 * expit(lam_lo * x_scaled) - 1)
    y_hi = beta_mean * (2 * expit(lam_hi * x_scaled) - 1)

    ax = axes[i]
    ax.plot(x_scaled, y_mean, color=CHANNEL_COLORS[ch], linewidth=2.5, label="Mean")
    ax.fill_between(
        x_scaled,
        y_lo,
        y_hi,
        alpha=0.2,
        color=CHANNEL_COLORS[ch],
        label="Confidence range",
    )

    # Mark current average spend level
    x_current = float(mean_weekly_spend[ch] / channel_max_spend[ch])
    y_current = beta_mean * (2 * expit(lam_mean * x_current) - 1)
    ax.axvline(x_current, color="black", linestyle="--", linewidth=1.5, alpha=0.7)
    ax.plot(
        x_current,
        y_current,
        "o",
        color="black",
        markersize=9,
        label="Current spend level",
        zorder=5,
    )

    # Determine zone: compute slope at current spend vs slope at 0.1 (early zone)
    s_current = expit(lam_mean * x_current)
    marginal_current = 2 * beta_mean * lam_mean * s_current * (1 - s_current)
    s_early = expit(lam_mean * 0.1)
    marginal_early = 2 * beta_mean * lam_mean * s_early * (1 - s_early)
    pct_of_early = (
        marginal_current / marginal_early * 100 if marginal_early > 0 else 100
    )

    if pct_of_early > 75:
        zone = "Early zone (steep)"
    elif pct_of_early > 40:
        zone = "Mid zone"
    else:
        zone = "Saturation zone (flat)"

    saturation_interpretation.append(
        {
            "Channel": ch,
            "Zone": zone,
            "Marginal return vs early (%)": f"{pct_of_early:.0f}%",
        }
    )

    ax.set_title(f"{ch}", fontsize=11, fontweight="bold", color=CHANNEL_COLORS[ch])
    ax.set_xlabel("Spend level\n(0 = none, 1 = max observed)")
    ax.set_ylabel("Contribution (scaled)" if i == 0 else "")
    if i == 0:
        ax.legend(fontsize=8, loc="upper left")

plt.suptitle(
    "Diminishing Returns Curves — Where Is Each Channel on the Curve?",
    fontsize=13,
    fontweight="bold",
    y=1.02,
)
plt.tight_layout()
plt.show()

print("Current spend zone by channel:")
print(pd.DataFrame(saturation_interpretation).set_index("Channel").to_string())
print()
print(
    "Marginal return vs early: 100% = same return as early spend; lower = more saturated."
)

The dashed vertical line marks where each channel currently sits on its response curve. Channels in the steep, early part of the curve have room to absorb more budget efficiently. Channels in the flat, upper part of the curve are showing diminishing returns — the next rupee spent there will generate less GMV than the same rupee would in a less saturated channel.

Note that the confidence ranges (shaded areas) reflect substantial uncertainty for all channels. The saturation curve shape is estimated from 12 months of data, which is not enough to pin it down precisely. These zones are directional signals, not definitive thresholds.

---
## 8. Budget Reallocation

Given the contribution estimates and diminishing returns analysis, we can compute how the current budget could be reallocated to maximise GMV while keeping total spend constant. The approach is to shift budget away from channels showing diminishing returns and towards channels that are still on the steep part of their response curves.

The metric used here is the **marginal return index**: the estimated incremental GMV per additional unit of spend at the current spend level. A higher index means the channel is still delivering strong returns on the next rupee.

In [ ]:
# Use saved ROI CSV if available, otherwise compute from model
if roi_csv is not None:
    roi_df = roi_csv.copy()
    print("Using saved ROI table from outputs/optimization/")
else:
    # Compute marginal returns and ROI from saturation parameters
    roi_rows = []
    for ch in config.channel_columns:
        lam = summary_lam.loc[f"saturation_lam[{ch}]", "mean"]
        beta = summary_beta.loc[f"saturation_beta[{ch}]", "mean"]

        x_cur = float(mean_weekly_spend[ch] / channel_max_spend[ch])

        # Contribution at current vs zero spend
        contrib_current = beta * (2 * expit(lam * x_cur) - 1)
        contrib_zero = beta * (2 * expit(0) - 1)
        incremental = contrib_current - contrib_zero

        spend_m = mean_weekly_spend[ch] / 1e6
        roi = incremental / spend_m if spend_m > 0 else 0.0

        # Marginal return at current spend
        s = expit(lam * x_cur)
        marginal = 2 * beta * lam * s * (1 - s)

        roi_rows.append(
            {
                "channel": ch,
                "mean_weekly_spend_INR": mean_weekly_spend[ch],
                "contribution_index": round(incremental, 4),
                "roi_per_1M_INR": round(roi, 4),
                "marginal_return": round(marginal, 4),
            }
        )

    roi_df = (
        pd.DataFrame(roi_rows)
        .set_index("channel")
        .sort_values("marginal_return", ascending=False)
    )
    print("Computed ROI from model parameters (no saved CSV found).")

print()
print("ROI index by channel (contribution index per 1M INR weekly spend):")
print(
    roi_df[
        ["mean_weekly_spend_INR", "contribution_index", "marginal_return"]
    ].to_string()
)

In [ ]:
marginal_col = (
    "marginal_return" if "marginal_return" in roi_df.columns else "roi_per_1M_INR"
)
roi_sorted = roi_df.sort_values(marginal_col, ascending=False)

fig, ax = plt.subplots(figsize=(9, 4))
bar_colors = [CHANNEL_COLORS.get(ch, "steelblue") for ch in roi_sorted.index]
roi_sorted[marginal_col].plot.bar(
    ax=ax, color=bar_colors, edgecolor="black", linewidth=0.5
)
ax.set_title(
    "Marginal Return Index per Channel\n(incremental GMV per additional unit of spend at current level)",
    fontsize=11,
    fontweight="bold",
)
ax.set_ylabel("Marginal return (contribution units)")
ax.tick_params(axis="x", rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Suggested reallocation: weight budget by marginal return
marginal_returns = roi_df[marginal_col]
weights = marginal_returns / marginal_returns.sum()
suggested_spend = weights * total_weekly_budget

# Apply light bounds: each channel keeps at least 10% of current and no more than 2x
lower_bounds = mean_weekly_spend * 0.10
upper_bounds = mean_weekly_spend * 2.0
suggested_spend = suggested_spend.clip(lower=lower_bounds, upper=upper_bounds)
# Renormalise to match total budget
suggested_spend = suggested_spend / suggested_spend.sum() * total_weekly_budget

realloc_df = pd.DataFrame(
    {
        "Current (INR/week)": mean_weekly_spend.round(0).astype(int),
        "Current Share (%)": spend_share,
        "Suggested (INR/week)": suggested_spend.round(0).astype(int),
        "Suggested Share (%)": (suggested_spend / suggested_spend.sum() * 100).round(1),
        "Change (INR/week)": (suggested_spend - mean_weekly_spend).round(0).astype(int),
    }
)

print("Current vs Suggested Reallocation (base scenario: same total budget):")
print(realloc_df.to_string())
print(f"\nTotal weekly budget: {total_weekly_budget:,.0f} INR (unchanged)")

**Important caveat:** These reallocation estimates are directional. They are derived from a model fitted on 12 months of data with monthly media spend distributed uniformly across weeks. The model cannot reliably distinguish the response curves of all four channels with this amount of data — the confidence ranges on the saturation curves are wide, and the channel rankings could change with more data.

Do not make large budget decisions based solely on these estimates. Validate the model on held-out data, collect at least two years of weekly observations, and consult with media planning experts before acting on specific channel allocation percentages.

---
## 9. Key Takeaways

**What we can say with confidence:**

- Sponsorship is the single largest driver of media spend at 46.6% of total budget, and the model confirms it is also the largest channel contributor to GMV in absolute terms — but spend share and contribution share may not be proportional.
- Digital and Online channels together account for 47.8% of spend and a significant share of modelled channel GMV, with different saturation and carryover profiles that suggest different roles in the media mix.
- TV is the smallest channel by spend (5.6%) but shows measurable contribution and potentially favourable marginal returns at its current spend level.
- The model's ability to reproduce observed weekly GMV (see the posterior predictive check in notebook 04) confirms that the channel structure and control variables are capturing the main drivers of sales variation.

**What the model suggests (directional):**

- TV shows a meaningful carryover effect relative to its spend level — campaigns may continue working for several weeks after they end, which is relevant for budget timing decisions.
- Sponsorship, as the highest-spend channel, may be approaching or entering the flatter part of its response curve — the next incremental rupee spent here may generate less GMV than the same rupee in a less saturated channel.
- The marginal return analysis suggests rebalancing some budget away from the most-saturated channel(s) towards channels still in the steep portion of their curves could improve total GMV without increasing overall spend.
- The contribution share vs spend share gap for one or more channels suggests the current allocation was not optimised on response-curve data — historical spend levels may reflect brand priorities or partnerships rather than marginal efficiency.

**What we need to verify before acting:**

- Collect at least two full years of weekly data (104+ observations) to give the model enough signal to reliably separate channel effects from seasonal patterns and one-off events.
- Validate model predictions on held-out months (e.g., fit on 10 months, predict the last 2) before trusting the response curves for budget decisions.
- Obtain daily-level media spend data if possible — pro-rata distributing monthly spend across weeks means all weeks in a month look identical to the model, limiting within-month attribution.
- Review the saturation and carryover estimates with the media buying team — the model's priors encode generic industry assumptions, and local market knowledge should inform whether those assumptions are appropriate for DT Mart.